In [3]:
import pandas as pd

pd.set_option('display.max_columns', None)

In [ ]:
push_opt = pd.read_parquet('../data/raw/3m/80464_push_opt_in.snappy.parquet')
push_inbox = pd.read_parquet('../data/raw/3m/80464_push_inbox_sent.snappy.parquet')
pd.read_parquet('../data/raw/3m/80464_open.snappy.parquet') #email

push_sent = pd.read_parquet('../data/raw/3m/80464_push_sent.snappy.parquet')
push_bounced = pd.read_parquet('../data/raw/3m/80464_push_bounced.snappy.parquet')
push_opened = pd.read_parquet('../data/raw/3m/80464_push_opened.snappy.parquet')

In [7]:
df_nps = pd.read_parquet('../data/stage/3m/customers_nps_churn_palive.parquet')

In [6]:
push_sent = pd.read_parquet('../data/raw/3m/80464_push_sent.snappy.parquet')
push_sent = push_sent.drop_duplicates(subset='EVENT_UUID')

In [7]:
push_opened = pd.read_parquet('../data/raw/3m/80464_push_opened.snappy.parquet')
push_opened = push_opened.drop_duplicates(subset=('PUSH_ID', 'LAUNCH_ID'))

In [8]:
push_bounced = pd.read_parquet('../data/raw/3m/80464_push_bounced.snappy.parquet')
push_bounced = push_bounced.drop_duplicates(subset=('PUSH_ID', 'LAUNCH_ID'))

In [18]:
renaming_columns = {
    'EVENT_CAPTURED_DT': 'BOUNCED_EVENT_CAPTURED_DT',
    'EVENT_UUID': 'BOUNCED_EVENT_UUID'
}
push_bounced = push_bounced.rename(columns=renaming_columns)
push_bounced.loc[:,'CUSTOMER_ID'] = pd.to_numeric(push_bounced['CUSTOMER_ID'], errors='coerce', downcast='integer')
push_bounced = push_bounced[push_bounced['CUSTOMER_ID'].notna()]
push_bounced.loc[:,'CUSTOMER_ID'] = push_bounced['CUSTOMER_ID'].astype('int')
push_bounced['PUSH_BOUNCED'] = 1

columns = [
    'RIID', 'BOUNCED_EVENT_UUID','LAUNCH_ID', 'PUSH_ID',
    'CUSTOMER_ID', 'BOUNCED_EVENT_CAPTURED_DT', 'PUSH_BOUNCED'
]
push_bounced = push_bounced[columns]

In [12]:
renaming_columns = {
    'EVENT_CAPTURED_DT': 'OPEN_EVENT_CAPTURED_DT',
    'EVENT_UUID': 'OPEN_EVENT_UUID'
}
push_opened = push_opened.rename(columns=renaming_columns)
push_opened.loc[:,'CUSTOMER_ID'] = pd.to_numeric(push_opened['CUSTOMER_ID'], errors='coerce', downcast='integer')
push_opened = push_opened[push_opened['CUSTOMER_ID'].notna()]
push_opened.loc[:,'CUSTOMER_ID'] = push_opened['CUSTOMER_ID'].astype('int')
push_opened['PUSH_OPEN'] = 1

columns = [
    'RIID', 'OPEN_EVENT_UUID','LAUNCH_ID', 'PUSH_ID',
    'CUSTOMER_ID', 'OPEN_EVENT_CAPTURED_DT', 'PUSH_OPEN'
]
push_opened = push_opened[columns]

In [13]:
push_sent.loc['CUSTOMER_ID'] = pd.to_numeric(push_sent['CUSTOMER_ID'], errors='coerce', downcast='integer')

In [22]:
push_sent_open = push_sent.merge(push_opened, on=('PUSH_ID', 'LAUNCH_ID'), how='left', suffixes=('', '_OPEN'))

In [28]:
push_sent_open_bounced = push_sent_open.merge(push_bounced, on=('PUSH_ID', 'LAUNCH_ID'), how='left', suffixes=('', '_BOUNCED'))

In [5]:
push_sent_open_bounced.head()

,EVENT_UUID,EVENT_TYPE_ID,ACCOUNT_ID,LIST_ID,RIID,CUSTOMER_ID,EVENT_CAPTURED_DT,EVENT_STORED_DT,CAMPAIGN_ID,LAUNCH_ID,PROGRAM_ID,PUSH_ID,USER_ID,APP_ID,PLATFORM_TYPE,year_ptt,month_ptt,day_ptt,RIID_OPEN,OPEN_EVENT_UUID,CUSTOMER_ID_OPEN,OPEN_EVENT_CAPTURED_DT,PUSH_OPEN,RIID_BOUNCED,BOUNCED_EVENT_UUID,CUSTOMER_ID_BOUNCED,BOUNCED_EVENT_CAPTURED_DT,PUSH_BOUNCED
0,e0d71544-4219-42f8-a78c-eaa6e5b115b0,501,80464,274885,268102245,3944947.0,30-Dec-2020 18:39:57,30-Dec-2020 18:40:02,587105,390645,587225,5F7BA137-F73B-43A1-BA9D-4DB248B0BF44,marceloteixeira1969@yahoo.com,5,ios,2020,12,30,None,None,NaN,None,NaN,None,None,NaN,None,NaN
1,8992391f-3f17-4671-a83b-33ddb1953b9b,501,80464,274885,310211905,3810353.0,30-Dec-2020 18:39:58,30-Dec-2020 18:40:02,587105,390645,587225,dad064f1-3f47-4049-a69f-a93812df33e4,ajennyfer.assis@gmail.com,5,android,2020,12,30,None,None,NaN,None,NaN,None,None,NaN,None,NaN
2,dac1bb54-4d42-40ce-b978-776c15e378b5,501,80464,274885,293235625,4112998.0,30-Dec-2020 18:39:57,30-Dec-2020 18:40:02,587105,390645,587225,3B72F41E-0CD4-416F-BD88-C48B664ADEAC,adrianampessotti@terra.com.br,5,ios,2020,12,30,None,None,NaN,None,NaN,None,None,NaN,None,NaN
3,b6cf63a4-fb31-45f2-b7a0-05c26f83ecca,501,80464,274885,279640545,2874405.0,30-Dec-2020 18:39:58,30-Dec-2020 18:40:02,587105,390645,587225,D5CB5665-BB8B-42F9-AE39-DC79F40CAC4C,souza.0003@hotmail.com,5,ios,2020,12,30,None,None,NaN,None,NaN,None,None,NaN,None,NaN
4,12490075-8057-4889-914f-7249bbc13936,501,80464,274885,307091645,4163664.0,30-Dec-2020 18:39:58,30-Dec-2020 18:40:02,587105,390645,587225,6bc6c2c9-d4ee-4afd-8ee5-48c746885206,passos_y@hotmail.com,5,android,2020,12,30,None,None,NaN,None,NaN,None,None,NaN,None,NaN


In [8]:
df_nps.head()

,consumer_id,role_id,cpf,email,birthdate,city,state,phonenumber,created_at,updated_at,eou_synced_at,last_activity_date,completed_at,paypal_linked_at,braintree_id,braintree_token,paypal_id,paypal_billing_agreement,paypal_email,paypal_cpf,paydiant_uri,paydiant_linked_at,stone_wallet_id,stone_linked_at,allow_newsletter,allow_promos,paypal_allow_automatic_payment,allow_automatic_payment,paypal_subscription_canceled,active,signup_complete,legacy_email,legacy_paypal_id,legacy_paypal_subscription,city_id,referral_code,mercadopago_linked_at,mercadopago_email,mercadopago_id,mfa_validated,country,nota_recomendacao_customer,nota_recomendacao_nps,nps,p_churn,p_currently_alive
0,3923437,7,1.022909e+10,brunoliebl7@gmail.com,1998-02-21,Rio Negrinho,SC,4.799791e+10,2020-04-06 17:48:24.000,2021-10-26 00:09:16.000,None,2021-10-26 00:09:16.000,2020-04-06 17:48:25.000,None,None,None,None,None,None,NaN,None,None,None,None,1.0,1,0,0,0,1,1,None,None,None,4631.0,BR58046,2020-12-06 22:30:54.000,brunoliebl7@gmail.com,615521117-BjnYel2QFjE9Mf,CONFIRMED,BRAZIL,10.0,NaN,1,0.162971,0.985849
1,3913125,7,2.561281e+09,ferueda76@gmail.com,1943-07-14,Arapoti,PR,4.199914e+10,2020-04-06 12:31:08.000,2021-10-27 21:56:53.000,None,2021-10-27 21:56:52.000,2020-04-06 12:31:08.000,None,None,None,None,None,None,NaN,None,None,None,None,1.0,1,0,0,0,1,1,None,None,None,2803.0,BTP8556,2020-04-06 12:35:55.000,ferueda76@gmail.com,542835804-lV3V7b2HrfTh72,CONFIRMED,BRAZIL,10.0,NaN,1,0.000000,NaN
2,3927102,7,1.910230e+10,marcelocipriani@gmail.com,1975-09-24,None,None,1.999714e+10,2020-04-06 21:30:03.000,2021-11-05 10:28:40.000,None,2021-11-05 10:28:39.000,2020-04-06 21:30:03.000,None,None,None,None,None,None,NaN,None,None,None,None,1.0,1,0,0,0,1,1,None,None,None,NaN,MI36905,None,None,None,CONFIRMED,BRAZIL,NaN,10.0,1,0.108373,0.989534
3,3923444,7,9.645964e+09,edkoby@gmail.com,1970-07-13,Balneário Camboriú,SC,4.798802e+10,2020-04-06 17:48:54.000,2021-11-09 00:37:24.000,None,2021-11-09 00:37:23.000,2020-04-06 17:48:55.000,None,None,None,None,None,None,NaN,None,None,None,None,1.0,1,0,0,0,1,1,None,None,None,4440.0,EK16749,2020-04-06 17:51:30.000,edkoby@gmail.com,542963388-gCYTHvk0bQTveN,CHANGED_DEVICE,BRAZIL,NaN,1.0,0,0.000000,0.653534
4,3928419,7,4.135720e+10,najla-andriele@hotmail.com,1992-11-29,Taubaté,SP,1.299150e+10,2020-04-06 23:11:19.000,2021-11-10 11:10:39.000,None,2021-11-10 11:10:38.000,2020-04-06 23:11:19.000,None,None,None,None,None,None,NaN,None,None,None,None,0.0,1,0,0,0,1,1,None,None,None,5312.0,NA58810,2021-05-03 12:40:57.000,najla-andriele@hotmail.com,665910526-esCcgR5izFZpOC,CONFIRMED,BRAZIL,NaN,10.0,1,0.327837,0.929038


In [14]:
nps_push_sent_open_bounced = pd.merge(df_nps, push_sent_open_bounced, left_on='consumer_id', right_on='CUSTOMER_ID', how='left', suffixes=('_nps', '_PUSHS'))

In [13]:
df_nps.shape

(5062, 46)

In [12]:
teste.shape

(99222, 74)

In [16]:
nps_push_sent_open_bounced.shape

(99222, 74)

#### Saving DataFrame

In [17]:
nps_push_sent_open_bounced.to_parquet('../data/stage/3m/customers_push.parquet')